# National ASTER Map of Australia <img align="right" src="../../resources/easi_logo.jpg">

#### Index
- [Overview](#Overview)
- [Setup (dask, imports, query)](#Setup)
- [Product definition (measurements, flags)](#Product-definition)
- [Quality layer (mask)](#Quality-layer)
- [Scaling and nodata](#Scaling-and-nodata)
- [Visualisation](#Visualisation)
- [Appendix](#Appendix)

## Overview

National ASTER Map of Australia comprises a set of 17 geoscience products made up of mosaiced ASTER (Advanced Spaceborne Thermal Emission and Reflection Radiometer) scenes across Australia. The individual geoscience products are a combination of bands and band ratios to highlight different mineral groups and parameters.

- CSIRO publication reference: https://doi.org/10.4225/08/584d948f9bbd1
- CSIRO data reference: https://doi.org/10.4225/08/51400D6F7B335
- Geoscience Australia catalogue: https://ecat.ga.gov.au/geonetwork/srv/eng/catalog.search#/metadata/74347
- NCI catalogue: https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f7754_6464_7646_8363

#### Data source and documentation

| Product name | Name and Metadata link | Interpretation |
|--|--|--|
| aster_aloh_group_composition | [AlOH group composition](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f1823_6337_2787_1326) | Blue is well ordered kaolinite, Al-rich white mica (muscovite, illite, paragonite), pyrophyllite, beidellite<br>Red is Al-poor (Si-rich) white mica (phengite), montmorillonite |
| aster_aloh_group_content | [AlOH group content](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f1168_3523_3375_4697) | Blue is low content<br>Red is high content |
| aster_false_colour | | Non-geological differences within and between ASTER scenes caused by green vegetation (red), fire scars, thin and thick cloud and cloud shadows. Use band 2 only for a gray-scale background to the content, composition and index colour products.
| aster_feoh_group_content | [FeOH group content](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f8711_1072_0979_6939) | Blue is low content<br>Red is high content |
| aster_ferric_oxide_composition | [Ferric oxide composition](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f7401_6489_2308_2725) | Blue-cyan is non-hematitie, i.e. goethite-rich<br>Red-yellow is hematite-rich |
| aster_ferric_oxide_content | [Ferric oxide content](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f0613_6306_9085_9820) | Blue is low abundance<br>Red is high abundance |
| aster_ferrous_iron_content_in_mgoh | [Ferrous iron content in MgOH/carbonate](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f2024_0154_4178_4072) | Blue is low ferrous iron content in carbonate and MgOH minerals like talc and tremolite<br>Red is high ferrous iron content in carbonate and MgOH minerals like chlorite and actinolite |
| aster_ferrous_iron_index | [Ferrous iron index](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f3532_2246_1929_4927) | Blue is low abundance<br>Red is high abundance |
| aster_green_vegetation | [Green vegetation content](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f7232_3072_6209_7100) | Blue is low content<br>Red is high content |
| aster_gypsum_index |
| aster_kaolin_group_index | [Kaolin group index](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f9063_0048_5432_4205) | Blue is low content<br>Red is high content |
| aster_mgoh_group_composition | [MgOH group composition](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f4127_0310_8856_3869) | Blue-cyan is magnesite-dolomite, amphibole, chlorite<br>Red is calcite, epidote, amphibole |
| aster_mgoh_group_content | [MgOH group content](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f1734_9663_3557_9883) | Blue is low content<br>Red is high content |
| aster_opaque_index | [Opaque index](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f3401_6961_4148_6080) | Blue is low content<br>Red is high content |
| aster_quartz_index | [TIR Quartz index](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f1011_7875_7538_2807)
| aster_regolith_ratios |
| aster_silica_index | [TIR Silica index](https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f2150_3654_7441_8491)

#### EASI pipeline

| Task | Summary |
|------|---------|
| Source | Index from https://data.dea.ga.gov.au/?prefix=ASTER_Geoscience_Map_of_Australia |
| Download | n/a |
| Preprocess | n/a |
| Format | n/a |
| Prepare | n/a |
| TODO | Consider reformatting the files and copying to EASI |

#### To Do
- [ ] Visualisation is very slow. Consider reformatting the files and copying to EASI
   - Closest issue/discussion https://discourse.holoviz.org/t/canvas-raster-failing-when-reading-with-dask/736
- [ ] Update colour limits for each product. Current limits from the CSIRO report do not seem to match the range in the GA files.

## Setup

#### Dask

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.0.92.156:36285")
client

#### Imports

In [ ]:
# Data tools
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime

# Datacube
import datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from odc.algo import xr_reproject   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_warp.py
from datacube.utils.geometry import GeoBox, box  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/geometry/_base.py
from datacube.utils.rio import configure_s3_access

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# import geoviews as gv
# from holoviews.operation.datashader import rasterize
hv.extension('bokeh', logo=False)

# Python
import sys, os, re

# Optional EASI tools
sys.path.append(os.path.expanduser('~/hub-notebooks/scripts'))
import notebook_utils

#### ODC database

In [ ]:
# For development products:
#  - This is a development ODC database while we test and demo this product.
# CONF = """
# [datacube]
# db_hostname: v2-db-easihub-csiro-eks.cluster-ro-cvaedcg0qvwd.ap-southeast-2.rds.amazonaws.com
# db_database: 
# db_username: 
# db_password: 
# """
# from datacube.config import read_config, LocalConfig
# dc = datacube.Datacube(config=LocalConfig(read_config(CONF)), env='datacube')

dc = datacube.Datacube()

#### Example query

Change any of the parameters in the query object below to adjust the location, time, projection, or spatial resolution of the returned datasets.

Use the Explorer interface to check the temporal and spatial coverage for each product:
- https://explorer.csiro.easi-eo.solutions  + /product (when available)

In [ ]:
# Area name
# min_longitude, max_longitude = (112.9172755, 153.6400543)   # Full coverage - very slow visualisation!
# min_latitude, max_latitude = (-43.7806434, -10.2856586)
# min_longitude, max_longitude = (112.5, 125)  # SW WA
# min_latitude, max_latitude = (-35, -25.5)
min_longitude, max_longitude = (119.5, 124)  # SW WA - mini test area
min_latitude, max_latitude = (-34, -31)

min_date = ''
max_date = ''
# product = 'aster_aloh_group_composition'
# product = 'aster_aloh_group_content'
# product = 'aster_false_colour'
# product = 'aster_feoh_group_content'
# product = 'aster_ferric_oxide_composition'
# product = 'aster_ferric_oxide_content'
product = 'aster_ferrous_iron_content_in_mgoh'
# product = 'aster_ferrous_iron_index'
# product = 'aster_green_vegetation'
# product = 'aster_gypsum_index'
# product = 'aster_kaolin_group_index'
# product = 'aster_mgoh_group_composition'
# product = 'aster_mgoh_group_content'
# product = 'aster_opaque_index'
# product = 'aster_quartz_index'
# product = 'aster_regolith_ratios'
# product = 'aster_silica_index'

query = {
    'product': product,                     # Product name
    'x': (min_longitude, max_longitude),    # "x" axis bounds
    'y': (min_latitude, max_latitude),      # "y" axis bounds
#     'time': (min_date, max_date),           # Any parsable date strings
}

native_res = (-0.000277777784500, 0.000277777784500)
native_crs = notebook_utils.mostcommon_crs(dc, query)
print(f'Native CRS: {native_crs}')

query.update({
    'output_crs': native_crs,               # EPSG code
    'resolution': native_res,             # Target resolution
    'group_by': 'solar_day',                # Scene ordering
    'dask_chunks': {'latitude': 2048, 'longitude': 2048},  # Dask chunks
})

In [ ]:
# Optional. Some products require AWS S3 credentials to supplied

# S3 credentials - required for s2_l2a
# configure_s3_access(aws_unsigned=True,requester_pays=False,client=client)
# print("Configured s3 requester pays data access")

In [ ]:
# Load data
data = dc.load(**query).astype('float')  # Convert to float64 to support NaNs

notebook_utils.heading(notebook_utils.xarray_object_size(data))
display(data)

# Calculate valid (not nodata) masks for each layer
valid_mask = masking.valid_data_mask(data)
notebook_utils.heading('Valid data masks for each variable')

## Product definition

Display the measurement definitions for the selected product.

Use `list_measurements` to show the details for a product, and `masking.describe_variable_flags` to show the flag definitions.

In [ ]:
# Measurement definitions for the selected product
measurement_info = dc.list_measurements().loc[query['product']]
notebook_utils.heading(f'Measurement table for product: {query["product"]}')
notebook_utils.display_table(measurement_info)

# Separate lists of measurement names and flag names
measurement_names = measurement_info[ pd.isnull(measurement_info.flags_definition)].index
flag_names        = measurement_info[pd.notnull(measurement_info.flags_definition)].index

notebook_utils.heading('Selected Measurement and Flag names')
notebook_utils.display_table(pd.DataFrame({
    'group': ['Measurement names', 'Flag names'],
    'names': [', '.join(measurement_names), ', '.join(flag_names)]
}))

# Flag definitions
for flag in flag_names:
    notebook_utils.heading(f'Flag definition table for flag name: {flag}')
    notebook_utils.display_table(masking.describe_variable_flags(data[flag]))

## Quality layer

In [ ]:
# No quality layer in the ASTER data

## Masking and Scaling

In [ ]:
# No scaling to be applied.

layer_name = 'Band_1'  # Applies to most products
if product == 'aster_false_colour':
    # Red: B3 361-4241
    # Green: B2 309-2913
    # Blue: B1 1-1961
    # Or Band 2 for grey scale
    layer_name = 'Band_2'  # Recommended band for grey scale. Alternatively, create a RGB image
    

# Apply valid mask and good pixel mask
layer = data[[layer_name]].where(valid_mask[layer_name])
layer = layer.persist()

## Visualisation

In [ ]:
# Generate a plot

options = {
    'title': f'{query["product"]}: {layer_name}',
    'width': 800,
    'height': 450,
    'aspect': 'equal',
    'cmap': cc.rainbow,
    'clim': (0, 255),                          # All products are scaled to 0-255
    'colorbar': True,
    'tools': ['hover'],
}

# Set the Dataset CRS
plot_crs = native_crs
if plot_crs in ('epsg:4326', 'EPSG:4283'):
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used

layer_plot = layer.hvplot.image(
    x = 'longitude', y = 'latitude',                        # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mean(),          # Datashader selects mean value
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                        # Dataset crs
    projection = ccrs.PlateCarree(),         # Output projection (use ccrs.PlateCarree() when coastline=True)
    coastline='10m',                         # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = options['clim'])

# display(layer_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(layer_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

## Appendix